In [16]:
import pandas as pd
import numpy as np
import tensorflow as tf

df = pd.read_csv(
    "https://www.dropbox.com/s/20yp4jzq7de0wzf/train.csv?dl=1"
)
print(df.head())

   id keyword location                                               text  \
0   1     NaN      NaN  Our Deeds are the Reason of this #earthquake M...   
1   4     NaN      NaN             Forest fire near La Ronge Sask. Canada   
2   5     NaN      NaN  All residents asked to 'shelter in place' are ...   
3   6     NaN      NaN  13,000 people receive #wildfires evacuation or...   
4   7     NaN      NaN  Just got sent this photo from Ruby #Alaska as ...   

   target  
0       1  
1       1  
2       1  
3       1  
4       1  


In [17]:
df_shuffled = df.sample(frac=1, random_state=123)

# Dropping id, keyword and location columns
df_shuffled.drop(["id", "keyword", "location"], axis=1, inplace=True)
df_shuffled.reset_index(inplace=True, drop=True)
print(df_shuffled.head())

                                                text  target
0  So you have a new weapon that can cause un-ima...       1
1  The f$&amp;@ing things I do for #GISHWHES Just...       0
2  DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...       1
3  Aftershock back to school kick off was great. ...       0
4  in response to trauma Children of Addicts deve...       0


In [18]:
test_df = df_shuffled.sample(frac=0.1, random_state=42)
train_df = df_shuffled.drop(test_df.index)
print(f"Using {len(train_df)} samples for training and {len(test_df)} for validation")

Using 6852 samples for training and 761 for validation


In [19]:
train_ds = train_df.to_numpy()
test_ds  = test_df.to_numpy()

In [23]:
for x, y in train_ds:
    print(x)
    print(y)
    break

So you have a new weapon that can cause un-imaginable destruction.
1


In [22]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [24]:
from transformers import BertTokenizer, TFBertForSequenceClassification

def convert_text_to_feature(review, tokenizer, max_length):  
    return tokenizer.encode_plus(review,
                                add_special_tokens = True,
                                max_length = max_length,
                                padding='max_length',
                                truncation=True,
                                return_attention_mask = True)

def map_feature_to_dict(input_ids, attention_masks, token_type_ids, label):
    return {"input_ids": input_ids,
            "token_type_ids": token_type_ids,
            "attention_mask": attention_masks,}, label

def encode_text(ds, tokenizer, max_length):
    input_ids_list = []
    token_type_ids_list = []
    attention_mask_list = []
    label_list = []
        
    for review, label in ds:
        bert_input = convert_text_to_feature(review, tokenizer, max_length)
    
        input_ids_list.append(bert_input['input_ids'])
        token_type_ids_list.append(bert_input['token_type_ids'])
        attention_mask_list.append(bert_input['attention_mask'])
        label_list.append([label])

    return tf.data.Dataset.from_tensor_slices(
                (input_ids_list, attention_mask_list, token_type_ids_list, label_list)).map(map_feature_to_dict)

In [25]:

max_length = 128
learning_rate = 2e-5
epochs = 3

model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name, do_lower_case=True)

In [26]:
tokenizer.encode_plus(x,
                      add_special_tokens = True,
                      max_length = max_length,
                      padding='max_length',
                      truncation=True,
                      return_attention_mask = True)

{'input_ids': [101, 2061, 2017, 2031, 1037, 2047, 5195, 2008, 2064, 3426, 4895, 1011, 10047, 22974, 22966, 6215, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [27]:
batch_size = 32

train_ds = encode_text(train_ds, tokenizer, max_length).shuffle(32).batch(batch_size)
val_ds   = encode_text(test_ds, tokenizer, max_length).batch(batch_size)

In [28]:
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
model.summary()

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________


In [29]:
model.layers[2].get_weights()

[array([[-0.00054472,  0.00559396],
        [-0.00767508, -0.0166137 ],
        [-0.01474272, -0.00928016],
        ...,
        [-0.01845066, -0.03318931],
        [ 0.0091162 ,  0.0120491 ],
        [-0.03723201, -0.01721322]], dtype=float32),
 array([0., 0.], dtype=float32)]

In [31]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=1e-08), 
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')]
)

history = model.fit(train_ds, epochs=epochs, validation_data=val_ds)

Epoch 1/3
215/215 [==============================] - 215s 794ms/step - loss: 0.4628 - accuracy: 0.7872 - val_loss: 0.3927 - val_accuracy: 0.8357
Epoch 2/3
215/215 [==============================] - 174s 809ms/step - loss: 0.3297 - accuracy: 0.8694 - val_loss: 0.4089 - val_accuracy: 0.8371
Epoch 3/3
215/215 [==============================] - 174s 808ms/step - loss: 0.2483 - accuracy: 0.9124 - val_loss: 0.4687 - val_accuracy: 0.8239
